In [28]:
import yfinance as yf
import ta
import pandas as pd
from datetime import date, timedelta

# ✅ Définir la période (exemple sur ~1 an daily)
end = date.today()
start = end - timedelta(days=365)

print(f"📥 Téléchargement des données de {start} à {end}...")

# ✅ Télécharger en daily (plus fiable et toujours dispo sur GC=F)
df = yf.download('GC=F', interval='1d', start=start, end=end)

if df.empty:
    print("❌ ERREUR : Yahoo n'a renvoyé aucune donnée pour cette période et cet intervalle.")
else:
    print("✅ Données reçues. Calcul des indicateurs techniques...")

    # ✅ Nettoyer
    df = df.dropna()

    # ✅ S'assurer que Close est une Series plate
    close_series = df['Close'].squeeze()

    # RSI
    df['rsi'] = ta.momentum.RSIIndicator(close_series, window=10).rsi()

    # EMA
    df['ema_9'] = close_series.ewm(span=9, adjust=False).mean()
    df['ema_21'] = close_series.ewm(span=21, adjust=False).mean()

    # MACD
    macd_calc = ta.trend.MACD(close_series)
    df['macd_line'] = macd_calc.macd()

    # Créer la target (hausse ou pas le lendemain)
    df['target'] = (df['Close'].shift(-1) > df['Close']).astype(int)

    # Nettoyer encore les NA introduites par indicateurs
    df = df.dropna()

    # ✅ Voir résultat final
    print(df.head())

    # ✅ Sauvegarder en CSV si tu veux
    df.to_csv('gold_features.csv')
    print("✅ Données sauvegardées dans gold_features.csv")


[*********************100%***********************]  1 of 1 completed

📥 Téléchargement des données de 2024-06-27 à 2025-06-27...
✅ Données reçues. Calcul des indicateurs techniques...
Price             Close         High          Low         Open Volume  \
Ticker             GC=F         GC=F         GC=F         GC=F   GC=F   
Date                                                                    
2024-08-02  2425.699951  2477.000000  2416.000000  2444.000000    550   
2024-08-05  2401.699951  2449.800049  2367.399902  2442.000000    334   
2024-08-06  2389.100098  2421.800049  2380.000000  2414.500000    163   
2024-08-07  2390.500000  2401.000000  2384.500000  2392.199951    130   
2024-08-08  2422.199951  2422.800049  2384.300049  2384.300049    201   

Price             rsi        ema_9       ema_21  macd_line target  
Ticker                                                             
Date                                                               
2024-08-02  58.297224  2411.014861  2397.236596  16.040555      0  
2024-08-05  50.585080  2409.1

In [34]:

df=pd.read_csv("gold_features.csv", sep=",",header=0, index_col=0)
df = df.dropna()
df.head(100)

,Close,High,Low,Open,Volume,rsi,ema_9,ema_21,macd_line,target
Price,,,,,,,,,,
2024-08-02,2425.699951171875,2477.0,2416.0,2444.0,550,58.297224,2411.014861,2397.236596,16.040555,0.0
2024-08-05,2401.699951171875,2449.800048828125,2367.39990234375,2442.0,334,50.585080,2409.151879,2397.642356,14.377284,0.0
2024-08-06,2389.10009765625,2421.800048828125,2380.0,2414.5,163,46.961167,2405.141523,2396.865787,11.905192,1.0
2024-08-07,2390.5,2401.0,2384.5,2392.199951171875,130,47.426126,2402.213218,2396.287079,9.944368,1.0
2024-08-08,2422.199951171875,2422.800048828125,2384.300048828125,2384.300048828125,201,56.926661,2406.210565,2398.642794,10.823556,1.0
...,...,...,...,...,...,...,...,...,...,...
2024-12-17,2644.39990234375,2652.300048828125,2632.5,2651.800048828125,705,46.344104,2662.463042,2659.538389,2.512917,0.0
2024-12-18,2636.5,2647.10009765625,2590.199951171875,2644.300048828125,566,44.665171,2657.270434,2657.443990,0.335790,0.0
2024-12-19,2592.199951171875,2610.300048828125,2582.10009765625,2582.10009765625,238,36.439806,2644.256337,2651.512714,-4.907670,1.0


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 226 entries, 2024-08-02 to 2025-06-26
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Close      226 non-null    object 
 1   High       226 non-null    object 
 2   Low        226 non-null    object 
 3   Open       226 non-null    object 
 4   Volume     226 non-null    object 
 5   rsi        226 non-null    float64
 6   ema_9      226 non-null    float64
 7   ema_21     226 non-null    float64
 8   macd_line  226 non-null    float64
 9   target     226 non-null    float64
dtypes: float64(5), object(5)
memory usage: 19.4+ KB


In [36]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle

# ✅ 2. Préparer features et target
X = df[['rsi', 'ema_9', 'ema_21', 'macd_line']]
y = df['target']

# ✅ 3. Split Train/Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ✅ 4. Entraîner RandomForest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# ✅ 5. Évaluer
y_pred = model.predict(X_test)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\n✅ Classification Report:\n", classification_report(y_test, y_pred))

# ✅ 6. Sauvegarder le modèle
with open('gold_signal_model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("✅ Modèle sauvegardé dans gold_signal_model.pkl")

✅ Accuracy: 0.5652173913043478

✅ Classification Report:
               precision    recall  f1-score   support

         0.0       0.47      0.47      0.47        19
         1.0       0.63      0.63      0.63        27

    accuracy                           0.57        46
   macro avg       0.55      0.55      0.55        46
weighted avg       0.57      0.57      0.57        46

✅ Modèle sauvegardé dans gold_signal_model.pkl


In [27]:

# ✅ Charger le modèle
with open('gold_signal_model.pkl', 'rb') as f:
    model = pickle.load(f)

print("✅ Modèle chargé avec succès !")

# ✅ Exemples de nouvelles données (RSI, EMA9, EMA21, MACD)
nouvelle_observation = {
    'rsi': 56.92,
    'ema_9': 2411.01,
    'ema_21': 2397.23,
    'macd_line': 16.04
}

X_new = pd.DataFrame([nouvelle_observation])

# ✅ Faire la prédiction
prediction = model.predict(X_new)[0]
proba = model.predict_proba(X_new)[0][1]

# ✅ Afficher le résultat
print("✅ Prédiction (0 = baisse, 1 = hausse):", prediction)
print(f"✅ Probabilité de hausse : {proba*100:.2f}%")


✅ Modèle chargé avec succès !
✅ Prédiction (0 = baisse, 1 = hausse): 1
✅ Probabilité de hausse : 69.00%
